In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/OpenClassrooms-Student-Center/4470406-utilisez-des-modeles-supervises-non-lineaire/main/data/ecg.csv", header=None)

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,1.0
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1.0
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1.0
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1.0
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1.0


In [4]:
#Je sélectionne toutes les colonnes sauf la dernière (qui est ma cible) pour les mettre dans la variable X

X = data.iloc[:,:-1]

In [5]:
#Je sélectionne la dernière colonne pour la mettre dans la variable Y

y = data.iloc[:, -1]

In [6]:
import plotly.graph_objects as pgo

In [7]:
#On vérifie que notre échantillon est bien équilibré


values = y.value_counts()/y.shape[0]

fig = pgo.Figure(data=[pgo.Pie(labels=['Sain','Malade'], values=values)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                  marker=dict(colors=['green', 'red'], line=dict(color='white', width=0.1)))
fig.update_layout(
    title_text="Distribution des ECG sains et malades",
    title_font_color="white",
    legend_title_font_color="yellow",
    paper_bgcolor="black",
    plot_bgcolor='black',
    font_color="white",
)
fig.show()

In [ ]:
#Notre échantillon est bien équilibré. Nous pouvons séparer nos données entre les entrées composées des valeurs
#des ECG et la sortie composée de "1" ou de "0" selon que l'ECG est sain ou malade

In [8]:
#Nous découpons nos données en un jeu d'entrainement et un jeu de test

from sklearn import model_selection
X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(X, y, test_size=0.3)

In [9]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers

In [10]:
#Nous allons créer notre premier réseau de neurones.

def initialize_model():

#Nous utilisons la fonction Sequential() qui est approprié pour notre problème. Nous ne développerons pas ici mais
#vous pouvez vous référer à la documentation de la bibliothèque Keras: https://keras.io/guides/sequential_model/
    model = models.Sequential()
#On indique à notre modèle la dimension des données d'entrées qui correspond au nombre de colonnes de X_train
    model.add(keras.Input(shape=(140)))
#On met la première couche de notre réseau de neurones. Nous avons une couche avec 140 perceptrons car nous avons
#140 colonnes dans nos données d'entrée comme dit plus haut. La fonction sigmoid est particulièrement indiqué
#pour les problèmes de classification. Nous vous encourageons à aller voir à quoi elle ressemble.
    model.add(layers.Dense(140, input_dim=2, activation='sigmoid'))
#Nous ajoutons une seconde couche car nous sommes dans un problème non linéaire comme dit dans le cours
    model.add(layers.Dense(1, activation='sigmoid'))

#Ici, nous pouvons ajouter des paramètres à notre modèle. Il faut juste retenir que "accuracy" permet d'avoir
#la précision de notre modèle et est particulièrement indiqué pour les problèmes de classification.
    model.compile(metrics=['accuracy'],
                  loss='binary_crossentropy',
                  optimizer='adam')

    return model

model = initialize_model()

In [11]:
#"summary" appliqué à notre modèle nous permet d'avoir les paramètres qui la compose ainsi que les dimensions de
#notre échantillon à la sortie de chaque couche

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 140)               19740     
                                                                 
 dense_1 (Dense)             (None, 1)                 141       
                                                                 
Total params: 19881 (77.66 KB)
Trainable params: 19881 (77.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model = initialize_model()

history = model.fit(X_train,
                    y_train,
                    epochs = 50,
                    batch_size = 4,
                    verbose = 1)

Epoch 1/50
875/875 [==============================] - 2s 2ms/step - loss: 0.0758 - accuracy: 0.9737
Epoch 2/50
875/875 [==============================] - 3s 3ms/step - loss: 0.0380 - accuracy: 0.9874
Epoch 3/50
875/875 [==============================] - 2s 3ms/step - loss: 0.0300 - accuracy: 0.9906
Epoch 4/50
875/875 [==============================] - 2s 2ms/step - loss: 0.0260 - accuracy: 0.9926
Epoch 5/50
875/875 [==============================] - 2s 2ms/step - loss: 0.0236 - accuracy: 0.9926
Epoch 6/50
875/875 [==============================] - 2s 2ms/step - loss: 0.0210 - accuracy: 0.9937
Epoch 7/50
875/875 [==============================] - 2s 2ms/step - loss: 0.0184 - accuracy: 0.9940
Epoch 8/50
875/875 [==============================] - 2s 2ms/step - loss: 0.0160 - accuracy: 0.9957
Epoch 9/50
875/875 [==============================] - 2s 3ms/step - loss: 0.0132 - accuracy: 0.9969
Epoch 10/50
875/875 [==============================] - 3s 3ms/step - loss: 0.0133 - accuracy: 0.9957

In [13]:
y_pred = model.predict(X_test)

47/47 [==============================] - 0s 1ms/step


In [14]:
#On peut évaluer notre modèle
result = model.evaluate(X_test, y_test, verbose=1)

# On peut extraire la "metrics" choisie
accuracy = result[1]

print(f"La précision de notre modèle est de {round(accuracy*100,2)} %")

47/47 [==============================] - 0s 2ms/step - loss: 0.0809 - accuracy: 0.9900
La précision de notre modèle est de 99.0 %
